In [1]:
import sys
sys.path.insert(0, '../pyAudioAnalysis-master')
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
from pydub import AudioSegment
import numpy as np
import librosa
import pickle 
import os

In [2]:
#path_to_files = '/home/lenovo/Prathvi/NeuralNetwork/Kannada_Dataset/5classsplit/'
#folder_path = '/home/lenovo/Prathvi/NeuralNetwork/Project/Kannadaextracted_Features/splitmonodata/'

path_to_files = '/home/lenovo/All5_KannadaDataAug2018/5Class/'
folder_path = '/home/lenovo/All5_KannadaDataAug2018/5Class_Extracted_features/'
max_classes = 5

#Dont Edit
max_classes += 1;

In [3]:
# Run this cell to freshly extract MFCC features for all Audio Clips
X_mfcc_detailed = []
y_mfcc_detailed = []

# Variables to Store Mfcc features
X_mfcc = []
y_mfcc = []

# Variables to Store Delta_Mfcc features
X_delta = []
y_delta = []

# Variables to Store Delta_Delta_Mfcc features
X_delta2 = []
y_delta2 = []

j_num  = 0

for i in range(1,max_classes):
        folder_name =  path_to_files + 'D' + str(i) + '/'
        path_list = os.listdir(folder_name)
        j_num = 0
        for j in path_list:
                j_num += 1
                if '.wav' in j:
                    y_, sr = librosa.load(folder_name + j)
                    
                            # Obtaining Delta-Mfcc features
                    try:
                        mfcc = librosa.feature.mfcc(y=y_, sr=sr)
                    except:
                        print ('Skipping this file ')
                        continue
                        
                    try:   # Obtaining Delta features from Mfcc
                            
                        delta = librosa.feature.delta(mfcc)
                    except:
                        print ('Skipping this file Delta features from Mfcc')
                        continue
                    
                    try:   # Obtaining Delta_Delta features from Mfcc
                            
                        delta2 = librosa.feature.delta(mfcc,order = 2)
                    except:
                        print ('Skipping this file Delta-Delta features from Mfcc')
                        continue
                        
                        
                    mfcc2 = np.array(mfcc)
                    X_mfcc_detailed_sub = []
                    y_mfcc_detailed_sub = []
                    for k in range(len(mfcc[0])):
                        X_mfcc_detailed_sub.append(mfcc[:,k].tolist())
                        X_mfcc_detailed_sub[k].append(j_num)
                        y_mfcc_detailed_sub.append(i)
                    X_mfcc_detailed.append(X_mfcc_detailed_sub)
                    y_mfcc_detailed.append(y_mfcc_detailed_sub)
                    
                    # Calculation of mean & SD are done Seperately bcoz shapes(vector_sizes) varies
                    # Mean & SD for MFCC features
                    mfcc_cumulative_features = []
                    for k in range(len(mfcc)):
                        mfcc_cumulative_features.append(mfcc[k].mean())
                        mfcc_cumulative_features.append(mfcc[k].std())
                    X_mfcc.append(mfcc_cumulative_features)
                    y_mfcc.append(i)
                    
                    # Mean & SD for Delta_MFCC_features Extraction
                    delta_cumulative_features = []
                    for k in range(len(delta)):
                        delta_cumulative_features.append(delta[k].mean())
                        delta_cumulative_features.append(delta[k].std())
                    X_delta.append(delta_cumulative_features)
                    y_delta.append(i)
                    
                    # Mean & SD for Delta-Delta_MFCC_features Extraction
                    delta2_cumulative_features = []
                    for k in range(len(delta2)):
                        delta2_cumulative_features.append(delta2[k].mean())
                        delta2_cumulative_features.append(delta2[k].std())
                    X_delta2.append(delta2_cumulative_features)
                    y_delta2.append(i)
                    
min_l = len(X_mfcc_detailed[0])
max_l = len(X_mfcc_detailed[0])
for i in range(len(X_mfcc_detailed)):
    if (len(X_mfcc_detailed[i]) < min_l):
        min_l = len(X_mfcc_detailed[i])
    
    if (len(X_mfcc_detailed[i]) > max_l):
        max_l = len(X_mfcc_detailed[i])
        
X_min_combined = []
y_min_combined = []

X_min_grouped = []
y_min_grouped = []
for i in range(len(X_mfcc_detailed)):
    X_min_combined.extend(X_mfcc_detailed[i][0:min_l])
    y_min_combined.extend(y_mfcc_detailed[i][0:min_l])
    X_min_grouped.append(X_mfcc_detailed[i][0:min_l])
    y_min_grouped.append(y_mfcc_detailed[i][0])
    


In [4]:
#Siva: for Storing MFCC features into CSV file format
with open(folder_path + 'MFCC_Features/MFCC_labels_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(y_mfcc,fp)
    
with open(folder_path +'MFCC_Features/MFCC_features_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(X_mfcc,fp)
    
with open(folder_path + 'MFCC_Features/MFCC_full_labels_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(y_min_combined,fp)
    
with open(folder_path +'MFCC_Features/MFCC_full_features_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(X_min_combined,fp)
    
with open(folder_path + 'MFCC_Features/MFCC_full_labels_grouped_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(y_min_grouped,fp)
    
with open(folder_path +'MFCC_Features/MFCC_full_features_grouped_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(X_min_grouped,fp)

In [5]:
#Siva: for Storing Delta-Mfcc into CSV file format
with open(folder_path + 'MFCC_Features/delta_labels_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(y_delta,fp)
    
with open(folder_path + 'MFCC_Features/delta_features_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(X_delta,fp)

In [6]:
#Siva: for Storing Delta-Delta-Mfcc features into CSV file format

with open(folder_path + 'MFCC_Features/delta2_labels_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(y_delta2,fp)
    
with open( folder_path + 'MFCC_Features/delta2_features_' + str(max_classes-1) + '.csv',mode='wb') as fp:
    pickle.dump(X_delta2,fp)
    


In [7]:
np.array(X_mfcc).shape

(198, 40)

In [8]:
X_mfcc

[[-226.68283782964627,
  51.187392022929011,
  161.19162746326097,
  27.272642681852574,
  -11.541066433643579,
  19.386550830998427,
  7.2246045164660568,
  18.880992361689621,
  -10.374296455594491,
  18.730274297188071,
  -4.3509242255190861,
  13.506832666938095,
  -18.146002560484437,
  13.627886825231974,
  -14.280018378398882,
  14.489194585802164,
  -11.506140508745874,
  9.9993474818023156,
  -8.4153234150766227,
  9.2789718560558434,
  -13.223518415852052,
  11.857719919097901,
  -8.2774524323504064,
  11.043039118795612,
  1.5204730651075651,
  9.0929798599417833,
  -11.110279414638502,
  8.963844507564831,
  0.89788063781771199,
  8.8086728446126568,
  -1.1413804646040326,
  7.5585859387088057,
  -6.0621575092304081,
  7.3297610995658884,
  -4.9628740115159014,
  6.6228784176252509,
  -4.7535785698314994,
  6.889004297642451,
  -3.4811324604465086,
  6.0847466974419753],
 [-278.30089976068587,
  51.434837589699335,
  108.110372942189,
  27.571730658623395,
  -23.72997716734